In [5]:
import pandas as pd
import numpy as np

# Définition des chemins de fichiers
chemin_fichier = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_nutriments_complet.csv"
chemin_sortie = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_scores_sante_normalises.csv"

def normaliser_valeurs_nutritionnelles(row):
    """Normalise les valeurs nutritionnelles pour une portion standard"""
    portions_standard = {
        'Viande': 200,
        'Fast-food': 300,
        'Fusion': 350,
        'Comfort-food': 300,
        'default': 300
    }
    
    portion = portions_standard.get(row['Catégorie'], portions_standard['default'])
    poids_total = sum([
        row['Protéines (g)'],
        row['Lipides (g)'],
        row['Glucides (g)']
    ])
    
    if poids_total == 0:
        return row
    
    facteur = portion / poids_total if poids_total > 0 else 1
    
    return pd.Series({
        'Protéines (g)': row['Protéines (g)'] * facteur,
        'Lipides (g)': row['Lipides (g)'] * facteur,
        'Glucides (g)': row['Glucides (g)'] * facteur,
        'Calories (kcal)': row['Calories (kcal)'] * facteur,
        'Fibres (g)': row['Fibres (g)'] * facteur,
        'Sucres (g)': row['Sucres (g)'] * facteur,
        'Sodium (mg)': row['Sodium (mg)'] * facteur
    })

def calculer_score_sante(row):
    """Calcule le score santé basé sur les valeurs nutritionnelles normalisées"""
    recommandations = {
        'Calories (kcal)': {'min': 300, 'max': 800, 'poids': 0.2},
        'Protéines (g)': {'min': 15, 'max': 30, 'poids': 0.25},
        'Lipides (g)': {'min': 10, 'max': 30, 'poids': 0.2},
        'Glucides (g)': {'min': 30, 'max': 90, 'poids': 0.15},
        'Fibres (g)': {'min': 4, 'max': 10, 'poids': 0.1},
        'Sodium (mg)': {'min': 200, 'max': 800, 'poids': 0.1}
    }
    
    scores = []
    
    for nutriment, limites in recommandations.items():
        valeur = row[nutriment]
        
        if pd.isna(valeur):
            scores.append(5 * limites['poids'])
            continue
        
        if nutriment == 'Sodium (mg)':
            if valeur < limites['min']:
                score = 8.5
            else:
                score = max(5, 10 - ((valeur - limites['min']) / (limites['max'] - limites['min'])) * 5)
        elif nutriment == 'Fibres (g)':
            score = min(10, (valeur / limites['max']) * 10)
        else:
            if valeur < limites['min']:
                score = (valeur / limites['min']) * 7.5
            elif valeur > limites['max']:
                score = max(0, 7.5 - ((valeur - limites['max']) / limites['max']) * 5)
            else:
                score = 7.5 + ((valeur - limites['min']) / (limites['max'] - limites['min'])) * 2.5
        
        scores.append(score * limites['poids'])
    
    return round(np.sum(scores), 1)

try:
    # Lecture du fichier avec gestion d'erreur
    print(f"Tentative de lecture du fichier : {chemin_fichier}")
    df = pd.read_csv(chemin_fichier)
    print("Fichier chargé avec succès!")

    # Normalisation des valeurs nutritionnelles
    colonnes_nutriments = ['Protéines (g)', 'Lipides (g)', 'Glucides (g)', 
                          'Calories (kcal)', 'Fibres (g)', 'Sucres (g)', 'Sodium (mg)']
    df[colonnes_nutriments] = df.apply(normaliser_valeurs_nutritionnelles, axis=1)

    # Calcul des scores
    df['Score_Sante'] = df.apply(calculer_score_sante, axis=1)

    def categoriser_score(score):
        if score >= 8:
            return 'Excellent'
        elif score >= 7:
            return 'Très sain'
        elif score >= 6:
            return 'Sain'
        elif score >= 5:
            return 'Moyen'
        elif score >= 4:
            return 'Peu sain'
        else:
            return 'À éviter'

    df['Categorie_Sante'] = df['Score_Sante'].apply(categoriser_score)

    # Affichage des résultats
    print("\nAnalyse nutritionnelle des plats (valeurs normalisées par portion) :")
    print("\nTop 5 des plats les plus sains :")
    print(df.sort_values('Score_Sante', ascending=False)[['Plat', 'Score_Sante', 'Categorie_Sante']].head())

    print("\nTop 5 des plats les moins sains :")
    print(df.sort_values('Score_Sante', ascending=True)[['Plat', 'Score_Sante', 'Categorie_Sante']].head())

    print("\nScore moyen par pays :")
    print(df.groupby('Pays')['Score_Sante'].mean().sort_values(ascending=False))

    # Sauvegarde des résultats
    df.to_csv(chemin_sortie, index=False)
    print(f"\nRésultats sauvegardés dans : {chemin_sortie}")

except FileNotFoundError:
    print(f"Erreur : Le fichier n'a pas été trouvé à l'emplacement : {chemin_fichier}")
    print("Veuillez vérifier le chemin du fichier et son existence.")
except Exception as e:
    print(f"Une erreur est survenue : {str(e)}")




Tentative de lecture du fichier : /Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET/plats_populaires_nutriments_complet.csv
Fichier chargé avec succès!

Analyse nutritionnelle des plats (valeurs normalisées par portion) :

Top 5 des plats les plus sains :
             Plat  Score_Sante Categorie_Sante
8         Lasagna          5.0           Moyen
21  Fried Chicken          4.6        Peu sain
16        Biryani          4.6        Peu sain
2     Ratatouille          4.5        Peu sain
15    Greek Salad          4.1        Peu sain

Top 5 des plats les moins sains :
                Plat  Score_Sante Categorie_Sante
17  Tandoori Chicken          0.9        À éviter
18       Green Curry          1.7        À éviter
14          Saganaki          2.2        À éviter
13          Tzatziki          2.4        À éviter
11          Moussaka          2.5        À éviter

Score moyen par pays :
Pays
ÉTATS-UNIS    3.900000
JAPON         3.700000
FRANCE     